In [ ]:
import os
from collections import defaultdict

import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
import pyomo.environ as pe
import pyomo.opt as po

In [ ]:
nodes_filename = os.path.join('data', 'nodes.txt')
edges_filename = os.path.join('data', 'edges.txt')
distance_filename = os.path.join('data', 'distance.csv')
pos_filename = os.path.join('data', 'pos.csv')

In [ ]:
nodes = []
with open(nodes_filename) as fh:
    for line in fh:
        nodes.append(int(line.strip()))
nodes

In [ ]:
edges = []
with open(edges_filename) as fh:
    for line in fh:
        edge = tuple(int(i) for i in line.strip().split(','))
        edges.append(edge)
edges

In [ ]:
df_distance = pd.read_csv(distance_filename, index_col=[0,1], header=0)
display(df_distance)
distance = df_distance['distance'].to_dict()
display(distance)

In [ ]:
df_pos = pd.read_csv(pos_filename, index_col=[0], header=0)
display(df_pos)
pos = {idx: tuple(v) for idx, v in df_pos.iterrows()}
display(pos)

In [ ]:
graph = nx.DiGraph()
graph.add_nodes_from(list(nodes))
graph.add_edges_from(list(edges))

In [ ]:
fig, ax = plt.subplots(figsize=(12, 12))
nx.draw_networkx_nodes(graph, pos=pos, ax=ax, node_color='lightgray',
                       edgecolors='black', node_size=2000)
nx.draw_networkx_labels(graph, pos=pos, ax=ax, labels=dict(zip(nodes, nodes)),
                        font_size=20)
nx.draw_networkx_edges(graph, pos=pos, ax=ax, node_size=2000, arrowsize=25)
nx.draw_networkx_edge_labels(graph, pos=pos, ax=ax, edge_labels=distance,
                             font_size=16, rotate=False)
plt.show()

In [ ]:
delta_neg = defaultdict(set)
delta_pos = defaultdict(set)
for (i, j) in edges:
    delta_neg[i].add(j)
    delta_pos[j].add(i)

In [ ]:
instance = pe.ConcreteModel()

In [ ]:
instance.nodes = pe.Set(initialize=nodes)
instance.s = 0
instance.t = 9
instance.edges = pe.Set(within=instance.nodes*instance.nodes, initialize=edges)

In [ ]:
instance.distance = pe.Param(instance.edges, initialize=distance)
instance.delta_neg = pe.Param(instance.nodes, initialize=delta_neg, within=pe.Any, default=set())
instance.delta_pos = pe.Param(instance.nodes, initialize=delta_pos, within=pe.Any, default=set())

In [ ]:
instance.x = pe.Var(instance.edges, domain=pe.Reals, bounds=(0, 1))

In [ ]:
def shortest_path(instance):
    return sum(instance.distance[i, j] * instance.x[i, j]
               for (i, j) in instance.edges)

instance.shortest_path = pe.Objective(sense=pe.minimize, rule=shortest_path)

In [ ]:
def flow_balance(instance, i):
    flow_in = sum([instance.x[j, i] for j in instance.delta_pos[i]])
    flow_out = sum([instance.x[i, j] for j in instance.delta_neg[i]])
    if i == instance.s:
        constraint = (flow_out == 1)
    elif i == instance.t:
        constraint = (flow_in == 1)
    else:
        constraint = (flow_in == flow_out)
    return constraint

instance.flow_balance = pe.Constraint(instance.nodes, rule=flow_balance)

In [ ]:
solver = po.SolverFactory('glpk')
result = solver.solve(instance)

In [ ]:
i = int(instance.s)
path_nodes = [i]
path_edges = []
stop = False
while not stop:
    for j in delta_neg[i]:
        if instance.x[i, j].value == 1:
            if j == int(instance.t):
                stop = True
            path_nodes.append(j)
            path_edges.append((i, j))
            i = j
            break

In [ ]:
node_colors = ['lightblue' if i in path_nodes else 'lightgray'
               for i in graph.nodes()]
edge_colors = ['blue' if (i, j) in path_edges else 'black'
               for (i, j) in graph.edges()]
edge_widths = [2 if (i, j) in path_edges else 1
               for (i, j) in graph.edges()]

In [ ]:
fig, ax = plt.subplots(figsize=(12, 12))

nx.draw_networkx_nodes(graph, pos=pos, ax=ax, node_color=node_colors,
                       edgecolors='black', node_size=2000)
nx.draw_networkx_labels(graph, pos=pos, ax=ax, labels=dict(zip(nodes, nodes)),
                        font_size=20)
nx.draw_networkx_edges(graph, pos=pos, ax=ax, node_size=2000, arrowsize=25,
                       edge_color=edge_colors, width=edge_widths)
nx.draw_networkx_edge_labels(graph, pos=pos, ax=ax, edge_labels=distance,
                             font_size=16, rotate=False)
nx.draw_networkx_edges
plt.show()